In [ ]:
import yfinance as yf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import os
file_path= "/content/drive/My Drive/Porfolio_Data/"
files = os.listdir(file_path)
files.sort()

In [ ]:
import csv
with open (file_path+files[-1], "r") as f:
  df=pd.read_csv(f)

In [ ]:
def get_analytics (stock_details_dict):
  results=[]
  data = yf.download(stock_details_dict["symbol"], period="400d", interval="1d", progress=False)
  if data.empty:
    raise Exception("No data found")
    return None
  else:
    closes = data['Close'].dropna()
    if len(closes)<365:
      return None
    else:
      closes = closes.sort_index(ascending=True)
      today_close = closes.iloc[-1]
      yesterday_close = closes.iloc[-2]
      close_7 = closes.iloc[-7]
      close_30 = closes.iloc[-30]
      close_90 = closes.iloc[-90]
      close_180 = closes.iloc[-180]
      close_365 = closes.iloc[-365]

      invested_amt_str = "0.0(0.0%)"
      if stock_details_dict['invested_amt'] > 0:
        invested_amt_str = str(stock_details_dict['invested_amt'])+'('+str(round(((float(today_close)*float(stock_details_dict['no_of_shares']))-float(stock_details_dict['invested_amt']))/float(stock_details_dict['invested_amt'])*100,2))+'%)'

      results.append({  "symbol": stock_details_dict["symbol"],
                      "today_close": round(float(today_close),2),
                      "yesterday_close": str(round(float(yesterday_close),2))+'('+str(round(((float(today_close)-float(yesterday_close))/float(yesterday_close))*100,2))+'%)',
                      "close_7": str(round(float(close_7),2))+'('+str(round(((float(today_close)-float(close_7))/float(close_7))*100,2))+'%)',
                      "close_30": str(round(float(close_30),2))+'('+str(round(((float(today_close)-float(close_30))/float(close_30))*100,2))+'%)',
                      "close_90": str(round(float(close_90),2))+'('+str(round(((float(today_close)-float(close_90))/float(close_90))*100,2))+'%)',
                      "close_180": str(round(float(close_180),2))+'('+str(round(((float(today_close)-float(close_180))/float(close_180))*100,2))+'%)',
                      "close_365": str(round(float(close_365),2))+'('+str(round(((float(today_close)-float(close_365))/float(close_365))*100,2))+'%)',
                      "invested_amt": invested_amt_str,
                      "average_buy_price": stock_details_dict['avg'],
                      "no_of_shares": stock_details_dict['no_of_shares']
                      })

  return results

In [ ]:
stock_details=[]
for i in df["Instrument"]:
  stock_details.append({"symbol": i+'.NS',
                        "avg": float(df[df['Instrument']==i]['Avg. cost'].values[0]),
                        "no_of_shares": int(df[df['Instrument']==i]['Qty.'].values[0]),
                        "invested_amt": float(df[df['Instrument']==i]['Invested'].values[0])})

results_df = pd.DataFrame(columns=['symbol', 'today_close', 'yesterday_close', 'close_7', 'close_30', 'close_90', 'close_180', 'close_365', 'invested_amt', 'average_buy_price', 'no_of_shares'])

for i in stock_details:
    analytics_result = get_analytics(i)
    if analytics_result:
        results_df = pd.concat([results_df, pd.DataFrame(analytics_result)], ignore_index=True)

display(results_df)

In [ ]:
def generate_stock_impact_analysis_prompt(today, ticker):
    """
    Generates a detailed prompt for Gemini API to analyze market data
    and its impact on a specific stock.
    """
    prompt = f""" Asking you on date {today}
    Act as a highly experienced financial analyst. Your task is to provide a concise, insightful analysis of the current market news and its potential impact on company having ticker as per yahoo finance** ({ticker})**.

    Focus specifically on the past 24-48 hours of significant news.

    Provide the analysis in two main sections:

    **1. Short-Term Impact Analysis (Next 1-7 Days):**
    - Summarize the 1-2 most critical recent news items or market developments directly impacting {ticker}.
    - Explain how these developments are likely to influence {ticker}'s stock price over the next 1 to 7 trading days.
    - Indicate the expected sentiment (positive, negative, neutral) and the primary reasons.
    - Mention any immediate catalysts or risks.

    **2. Long-Term Impact Analysis (Next 1-6+ Months):**
    - Identify any current trends, macroeconomic factors, industry-specific shifts, or regulatory changes that, based on recent news, could have a sustained impact on {ticker}'s performance over the next 1 month to 6+ months.
    - Explain the potential long-term implications (e.g., changes in revenue streams, market position, competitive landscape, valuation multiples).
    - Provide a high-level outlook (positive, negative, neutral) for the stock's long-term trajectory based on these factors.

    **Output Format:**
    Please use clear headings and bullet points for readability. Avoid jargon where simpler terms suffice. Keep the response to a maximum of 400 words.

    **Example Context (Do not use this for analysis, this is just for your understanding):**
    If there's news about a new product launch, discuss its potential to increase short-term sales and long-term market share. If there's news about rising interest rates, discuss its impact on the company's borrowing costs (short-term) and valuation (long-term).
    """
    return prompt

In [ ]:
import json
import requests
from google.colab import userdata # For securely accessing your API key
import datetime

CURRENT_DATE = datetime.date.today()
latest_stock_news= pd.DataFrame(columns=['symbol', 'news'])

for i in results_df['symbol']:
  API_KEY = userdata.get('GOOGLE_API_KEY') # Get from Colab Secrets

  model_id = 'gemini-2.0-flash'
  url = f"https://generativelanguage.googleapis.com/v1beta/models/{model_id}:generateContent?key={API_KEY}"

  headers = {
      "Content-Type": "application/json",
  }

  payload = {
      "contents": [
          {
              "parts": [
                  {"text": generate_stock_impact_analysis_prompt(CURRENT_DATE, i)}
              ]
          }
      ],
      "generationConfig": {
          "temperature": 0.5,
          "maxOutputTokens": 500
      },
      "safetySettings": [
          {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
          {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
          {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
          {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
      ]
  }

  try:
      response = requests.post(url, headers=headers, json=payload)
      response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

      response_json = response.json()
      if 'candidates' in response_json and len(response_json['candidates']) > 0:
          generated_text = response_json['candidates'][0]['content']['parts'][0]['text']
          print(f"\n--- AI Generated Text for---{i}")
          print(generated_text)
          latest_stock_news= pd.concat([latest_stock_news, pd.DataFrame([{'symbol': i, 'news': generated_text}])], ignore_index=True)
      else:
          print("\n--- No AI Generated Content Found ---")

  except requests.exceptions.HTTPError as e:
      print(f"HTTP Error: {e}")
      print(f"Response Body: {e.response.text}")
  except requests.exceptions.ConnectionError as e:
      print(f"Connection Error: {e}")
  except requests.exceptions.Timeout as e:
      print(f"Timeout Error: {e}")
  except requests.exceptions.RequestException as e:
      print(f"An error occurred: {e}")
  except json.JSONDecodeError:
      print(f"Failed to decode JSON from response: {response.text}")

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from google.colab import userdata

# Email configuration
sender_email = 'your_name@gmail.com'
receiver_email = 'your_name@gmail.com'
# Get the App Password from Colab secrets
password = userdata.get('GMAIL_APP_PASSWORD')


# Create the email message
message = MIMEMultipart()
message['From'] = sender_email
message['To'] = receiver_email
message['Subject'] = 'Portfolio Analysis Results'

# Convert DataFrame to HTML
html1 = results_df.to_html()
html2 = latest_stock_news.to_html()

# Attach the HTML to the email
message.attach(MIMEText(html1, 'html'))
message.attach(MIMEText(html2, 'html'))

# Send the email
try:
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(sender_email, password)
    text = message.as_string()
    server.sendmail(sender_email, receiver_email, text)
    server.quit()
    print('Email sent successfully!')
except Exception as e:
    print(f"Failed to send email: {e}")